In [2]:
import pymc as pm
import numpy as np

num_samples = 100
num_features = 10

X_matrix = np.zeros((num_samples, num_features))
target_y = np.ones(num_samples)

with pm.Model() as logistic_regression_model:
    coeffs_prior = pm.MvNormal('coeffs_prior', mu=np.zeros(num_features), cov=np.identity(num_features), shape=num_features)
    lin_predictor = pm.math.dot(X_matrix, coeffs_prior)
    probabilities = pm.math.sigmoid(lin_predictor)
    y_observed = pm.Bernoulli('y_observed', p=probabilities, observed=target_y)
    
with logistic_regression_model:
    posterior_samples = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [coeffs_prior]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 3 seconds.


In [ ]:
import pymc as pm
import numpy as np
import arviz as az

np.random.seed(365)

sample_size = 100
feature_count = 1
true_coefficient = 2.5
noise_level = 1

X_data = np.random.normal(size=(sample_size, feature_count))
mean_response = X_data * true_coefficient
response_y = np.random.normal(mean_response.flatten(), scale=noise_level)

with pm.Model() as robust_linear_model:
    coefficient_prior = pm.Normal('coefficient', mu=0, sigma=10)
    degrees_of_freedom = pm.Exponential('degrees_of_freedom', 1/29) + 1
    precision_prior = pm.Gamma('precision_prior', alpha=degrees_of_freedom/2, beta=degrees_of_freedom/2, shape=sample_size)
    sigma = pm.Deterministic('sigma', 1 / np.sqrt(precision_prior))
    observed_y = pm.Normal('observed_y', mu=X_data[:, 0] * coefficient_prior, sigma=sigma, observed=response_y)
    posterior_trace = pm.sample(1000, target_accept=0.9, return_inferencedata=True)

precision_posterior_mean = az.summary(posterior_trace, var_names=['precision_prior'])['mean'].values
outlier_threshold = np.quantile(precision_posterior_mean, 0.05)
identified_outliers = np.where(precision_posterior_mean < outlier_threshold)[0]

print(f"Identified potential outliers: {identified_outliers}")